In [1]:
from mxnet import gluon
from mxnet import nd
from mxnet import autograd
def transform(data,label):
    return data.astype('float32')/255,label.astype('float32')
minist_train=gluon.data.vision.FashionMNIST(root='fashion-mnist/',train=True,transform=transform)
minist_test=gluon.data.vision.FashionMNIST(root='fashion-mnist/',train=False,transform=transform)
batch_size=256 
train_data=gluon.data.DataLoader(minist_train,batch_size=batch_size,shuffle=True)
test_data=gluon.data.DataLoader(minist_test,batch_size=batch_size,shuffle=True)

# 两层隐含层

In [2]:
from mxnet import gluon
net=gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(256,activation='relu'))
    net.add(gluon.nn.Dense(10))
net.initialize()
softmax_crossEntropy=gluon.loss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':.5})

In [3]:
def accuracy(output,label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()
def evaluate_accuracy(test_data,net):
    acc=0. 
    for data,label in test_data:
        output=net(data)
        acc+=accuracy(output,label)
    return acc/len(test_data)


In [4]:
from mxnet import autograd
epochs = 4
for e in range(1, epochs+1):
    train_acc, train_loss = 0., 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_crossEntropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        train_acc += accuracy(output, label)
        train_loss += nd.mean(loss).asscalar()
    test_acc = evaluate_accuracy(test_data, net)
    print('epoch %d,loss:%f..,train_accuracy:%f,test_accuracy:%f..' % (e, train_loss/len(train_data), train_acc/len(train_data),
                                                                       test_acc/len(test_data)))

epoch 1,loss:0.722239..,train_accuracy:0.737295,test_accuracy:0.020984..
epoch 2,loss:0.467453..,train_accuracy:0.827521,test_accuracy:0.021455..
epoch 3,loss:0.410668..,train_accuracy:0.850421,test_accuracy:0.021707..
epoch 4,loss:0.378833..,train_accuracy:0.860029,test_accuracy:0.021685..


In [5]:
dense=net[1]
dense.weight.data(),dense.bias.data()

(
 [[ 0.00679919  0.01292737  0.03033251 ... -0.02541603 -0.0150122
    0.05347459]
  [ 0.06538177  0.05608409  0.00654458 ... -0.01343163  0.02962739
    0.04782937]
  [ 0.05228012 -0.06844398  0.06559557 ... -0.05495931  0.00956061
   -0.04069941]
  ...
  [-0.06626982 -0.0615694   0.04073513 ...  0.02512913 -0.0019495
    0.01449291]
  [ 0.04740904  0.01716206  0.02910219 ...  0.03891923  0.06823969
    0.03291409]
  [ 0.03623506  0.01020497 -0.04744481 ... -0.03525304 -0.01251852
   -0.06929076]]
 <NDArray 256x784 @cpu(0)>, 
 [ 2.16267973e-01  2.10423648e-01 -3.91482096e-03  1.48102388e-01
   5.17111719e-02  3.33081037e-02 -3.59931886e-02  2.29611881e-02
   1.06728747e-01  9.28028896e-02  2.96972245e-01  9.33107436e-02
   2.91183833e-02  4.90503348e-02  1.36192217e-01  1.85357884e-01
   9.90470685e-03 -3.22347833e-03  4.41323640e-03 -2.76640933e-02
   1.60672572e-02  3.04576885e-02  1.47342468e-02  1.05687585e-02
   1.37642741e-01 -9.56155267e-03 -1.32239079e-02 -1.24927564e-02
   1